### Импорт библиотек

In [1]:
import pandas as pd
import io
import re
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
import glob
from collections import Counter

ModuleNotFoundError: No module named 'pdfminer'

In [ ]:
df = pd.read_json

### Считование pdf файлов и их парсинг

In [2]:
# Функция считывания pdf файлов
def extract_text_from_pdf(pdf_path):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle)
    page_interpreter = PDFPageInterpreter(resource_manager, converter)
 
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)
 
        text = fake_file_handle.getvalue()

    converter.close()
    fake_file_handle.close()
 
    if text:
        return text

In [3]:
# Функция парсинга текста
def pars_pdf(pdf, text):
    text = text.replace("Copyright © Союз «Ворлдскиллс Россия»", "") 
    x = text.split('Раздел Важность (%)')[1].split('Всего 100')[0]
    
    Ymenia = x.split('Специалист должен знать и понимать:')
    Ymenia.pop(0)
    for i, Y in enumerate(Ymenia):
        Ymenia[i] = Y.split("Специалист должен уметь:")[0]
        
    Preg = r'\d{1,2}.{3,100}\d{1,2}'
    Pokazatels = re.findall(Preg, x)
    Prosents = []
    for P in Pokazatels:
        Prosents.append(P.split(' ')[-1])
    for i, P in enumerate(Pokazatels):
        Pokazatels[i] = re.sub(r"\d+","",P)
    
    Pokazatel = []
    for i in range(len(Ymenia)):
        Pokazatel.append(Pokazatels[i])
    Prosent = []
    for i in range(len(Ymenia)):
        Prosent.append(Prosents[i])
        
    Kompet = []
    for i in range(len(Pokazatel)):
        Kompet.append(pdf)
    return Kompet, Pokazatel, Prosent, Ymenia

In [4]:
# загрузка в list пути всех нужных pdf файлов
all_pdf = glob.glob(r"C:\Users\Данил\Desktop\Lkz\Lkz\Data\Технические описания компетенций\*.pdf")

In [5]:
len(all_pdf)

30

In [6]:
# Перебор текста из pdf файла, парсинг
Brak = []
Kompet = []
Pokazatel = []
Prosent = []
Ymenia = []
for pdf in all_pdf:
    try:  
        x = extract_text_from_pdf(pdf)
        print(pdf.split('\\')[-1])
        Ko, Po, Pr, Ym = pars_pdf(pdf.split('\\')[-1], x)
        Kompet += Ko
        Pokazatel += Po
        Prosent += Pr
        Ymenia += Ym   
    except:
        Brak.append(pdf)

Магистральные линии связи. Строительство и эксплуатация ВОЛП.pdf
Медицинская оптика.pdf
Метрология.pdf
Мехатроника .pdf
Мобильная робототехника.pdf
Обработка листового металла.pdf
Обслуживание грузовой техники.pdf
Обслуживание и ремонт оборудования релейной защиты и автоматики.pdf
Огранка ювелирных вставок.pdf
Организация строительного производства.pdf
Охрана окружающей среды.pdf
Охрана труда.pdf
Плотницкое дело(юниоры).pdf
Плотницкое дело.pdf
Полимеханика и автоматизация.pdf
Правоохранительная деятельность (Полицейский).pdf
Преподавание музыки в школе.pdf
Производственная сборка изделий авиационной техники.pdf
Производство металлоконструкций.pdf
Промышленный дизайн .pdf
Сметное дело.pdf
Спасательные работы.pdf
Столярное дело.pdf
Туроператорская деятельность.pdf
Укладка напольных покрытий.pdf
Управление локомотивом.pdf
Управление экскаватором.pdf
Холодильная техника и системы кондиционирования(юниоры).pdf
Холодильная техника и системы кондиционирования.pdf
Экспедирование грузов.pdf


In [65]:
# Вывод файлов не подлежащих автоматической обработке 
Brak

['C:\\Users\\Данил\\Desktop\\Lkz\\Lkz\\Data\\Технические описания компетенций\\Магистральные линии связи. Строительство и эксплуатация ВОЛП.pdf',
 'C:\\Users\\Данил\\Desktop\\Lkz\\Lkz\\Data\\Технические описания компетенций\\Мехатроника .pdf',
 'C:\\Users\\Данил\\Desktop\\Lkz\\Lkz\\Data\\Технические описания компетенций\\Обработка листового металла.pdf',
 'C:\\Users\\Данил\\Desktop\\Lkz\\Lkz\\Data\\Технические описания компетенций\\Охрана труда.pdf',
 'C:\\Users\\Данил\\Desktop\\Lkz\\Lkz\\Data\\Технические описания компетенций\\Полимеханика и автоматизация.pdf',
 'C:\\Users\\Данил\\Desktop\\Lkz\\Lkz\\Data\\Технические описания компетенций\\Правоохранительная деятельность (Полицейский).pdf',
 'C:\\Users\\Данил\\Desktop\\Lkz\\Lkz\\Data\\Технические описания компетенций\\Сметное дело.pdf',
 'C:\\Users\\Данил\\Desktop\\Lkz\\Lkz\\Data\\Технические описания компетенций\\Спасательные работы.pdf',
 'C:\\Users\\Данил\\Desktop\\Lkz\\Lkz\\Data\\Технические описания компетенций\\Экспедирование гру

### Создание DataFrame и добавление туда полученных данных

In [7]:
# Создание DataFrame
df = pd.DataFrame({
    'Компетенция': [],
    'Показатель': [],
    'Процент': [],
    'Умения': []
})

In [106]:
df

,Компетенция,Показатель,Процент,Умения


In [69]:
print(len(Kompet), len(Pokazatel), len(Prosent), len(Ymenia))

114 114 114 114


In [8]:
df['Компетенция'] = Kompet

In [9]:
df['Показатель'] = Pokazatel

In [10]:
df['Процент'] = Prosent

In [11]:
df["Умения"] = Ymenia

In [111]:
# Просмотр первых 10 строк
df.head(10)

,Компетенция,Показатель,Процент,Умения
0,Медицинская оптика.pdf,"Организовывать, планировать и анализировать р...",7," назначение, использование, техническое обсл..."
1,Медицинская оптика.pdf,Организовывать сбыт продукции,6,• основные торговые марки производителей меди...
2,Медицинская оптика.pdf,Устанавливать межличностную коммуникацию,12,"• основы психологии общения; • типы, характер..."
3,Медицинская оптика.pdf,Проводить продажу (или продавать ),6,Медицинская оптика 8 ...
4,Медицинская оптика.pdf,Проводить измерения и контроль параметров .,47.5,• классификацию линз; • классификацию оправ; ...
5,Медицинская оптика.pdf,Обрабатывать линзы и собирать очки .,18.5,• способы проверки линз на оптических прибора...
6,Медицинская оптика.pdf,"Принимать и применять решения по ремонту, зам...",3,• технологии и методы ремонта оправ корригиру...
7,Метрология.pdf,Соблюдение общих требований техники безопасно...,10,"• Нормы техники безопасности ; • Безопасное, ..."
8,Метрология.pdf,Обработка результатов измерения,30,"• Законодательство Российской Федерации, регл..."
9,Метрология.pdf,Оформление и ведение производственно-техничес...,20,"• Законодательство Российской Федерации, регл..."


### выгрузка данных из файла csv, проверка корректности перенесения данных

In [12]:
df2 = pd.read_csv(r"C:\Users\Данил\Desktop\Lkz\Lkz\Data\dataset.csv", sep = ";", encoding="windows-1251", header=None)
df2.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,Разработка компьютерных игр и мультимедийных п...,Разработка компьютерных игр и мультимедийных п...,Организация и управление работой,13,Специалист должен знать и понимать: • Принципы...,Программирование на языке С#,27,Специалист должен знать и понимать: • Базовые ...,Unity 3D,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Разработка мобильных приложений(юниоры),Смартфоны продаются и продажи только растут. К...,Организация и управление работой,5,Специалист должен знать и понимать: • Принципы...,"Решение проблем, инновационность и инициативность",5,Специалист должен знать и понимать: • Общие ти...,Анализ и проектирование мобильного приложения,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Преподавание в младших классах,Высококвалифицированные специалисты в сфере на...,Общекультурное развитие,10,Специалист должен знать и понимать:\n• Основы ...,Культура безопасного труда Организация рабочег...,5,Специалист должен знать и понимать: Участник к...,Общепрофессиональное развитие,40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Дошкольное воспитание(юниоры),Воспитатель детей дошкольного возраста осущес...,Соблюдение санитарных норм и правил профилакти...,16,Специалист должен знать и понимать:\n• Стандар...,"Поиск, анализ и оценка информации, необходимой...",24,Специалист должен знать и понимать: • основную...,Использование информационно-коммуникационных т...,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Дошкольное воспитание,Воспитатель детей дошкольного возраста осущест...,Соблюдение санитарных норм и правил профилакти...,16,Специалист должен знать и понимать: • Стандарт...,"Поиск, анализ и оценка информации, необходимой...",24,Специалист должен знать и понимать:\n• основну...,Использование информационно-коммуникационных т...,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
df2.shape

(130, 32)

### Формирование структуры набора данных

In [14]:
df2_dop = pd.DataFrame(columns=['0', '1', '2', '3', '4'])

In [15]:
# Цикл преобразует структуру в нужный нам вид
for i in range(df2.shape[0]):
    j=2
    while j < df2.shape[1]:
        df2_dop.loc[len(df2_dop)] = [df2.loc[i, 0], df2.loc[i, j], df2.loc[i, j+1], df2.loc[i, j+2]]
        j+=3

In [16]:
df2_dop.columns = ['Компетенция', 'описание', 'Показатель', 'Процент', 'Умения']

In [113]:
df2_dop

,Компетенция,Показатель,Процент,Умения
0,Разработка компьютерных игр и мультимедийных п...,Организация и управление работой,13,Специалист должен знать и понимать: • Принципы...
1,Разработка компьютерных игр и мультимедийных п...,Программирование на языке С#,27,Специалист должен знать и понимать: • Базовые ...
2,Разработка компьютерных игр и мультимедийных п...,Unity 3D,30,Специалист должен знать и понимать: • Основные...
3,Разработка компьютерных игр и мультимедийных п...,Оптимизация компьютерных игр и приложений,10,Специалист должен знать и понимать: • Особенно...
4,Разработка компьютерных игр и мультимедийных п...,Анализ и проектирование,15,Специалист должен знать и понимать: • Важность...
...,...,...,...,...
1295,Ювелирное дело,Изготовление сложных элементов и ювелирных изд...,20,Специалист должен знать и понимать: • Способы ...
1296,Ювелирное дело,Обработка поверхности,15,Специалист должен знать и понимать: • Специали...
1297,Ювелирное дело,NaN,NaN,NaN
1298,Ювелирное дело,NaN,NaN,NaN


### Получение итоговой таблицы

In [17]:
# Объединение таблиц
data = pd.concat([df2_dop, df])

In [234]:
data.head(15)

,Компетенция,Показатель,Процент,Умения
0,Разработка компьютерных игр и мультимедийных п...,Организация и управление работой,13,"• Принципы и навыки, обеспечивающие продуктив..."
1,Разработка компьютерных игр и мультимедийных п...,Программирование на языке С#,27,• Базовые алгоритмы программирования • Структ...
2,Разработка компьютерных игр и мультимедийных п...,Unity 3D,30,• Основные принципы и понятия разработки игр ...
3,Разработка компьютерных игр и мультимедийных п...,Оптимизация компьютерных игр и приложений,10,• Особенности оптимизации компьютерных игр и ...
4,Разработка компьютерных игр и мультимедийных п...,Анализ и проектирование,15,• Важность принятия во внимание всех возможны...
5,Разработка компьютерных игр и мультимедийных п...,Публикация,5,• Особенности сборки приложения под разные ОС...
6,Разработка компьютерных игр и мультимедийных п...,NaN,NaN,NaN
7,Разработка компьютерных игр и мультимедийных п...,NaN,NaN,NaN
8,Разработка компьютерных игр и мультимедийных п...,NaN,NaN,NaN
9,Разработка компьютерных игр и мультимедийных п...,NaN,NaN,NaN


In [18]:
data = data.dropna()
data = data.reset_index(drop=True)

In [19]:
data["Умения"] = data["Умения"].str.replace("Специалист должен знать и понимать:", "")

In [20]:
data.to_csv('C:\\Users\\Данил\\Desktop\\data_ws.csv', index=False)

### Создание словаря

In [197]:
all_w = list(data["Умения"])

In [198]:
all_w = str(all_w)

In [199]:
all_w = all_w.replace('•','')
all_w = all_w.replace('\\','')

In [145]:
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords

In [146]:
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+" # удаляет всю не кирилицу 
stopwords_ru = stopwords.words("russian") # удаляем СТОП-СЛОВА
morph = MorphAnalyzer() # используется для нахождения нормальной формы слова
# функция лематизации 
def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]           
            tokens.append(token)
    return tokens

In [200]:
S = lemmatize(all_w)

In [201]:
S

['принцип',
 'навык',
 'обеспечивать',
 'продуктивный',
 'работа',
 'преимущество',
 'использование',
 'современный',
 'средство',
 'разработка',
 'специалист',
 'должный',
 'уметь',
 'использовать',
 'предоставить',
 'ресурс',
 'продуктивный',
 'работа',
 'применять',
 'исследовательский',
 'технология',
 'навык',
 'иметь',
 'представление',
 'самый',
 'последний',
 'отраслевой',
 'рекомендация',
 'анализировать',
 'результат',
 'собственный',
 'деятельность',
 'сравнение',
 'ожидание',
 'потребность',
 'клиент',
 'организация',
 'базовый',
 'алгоритм',
 'программирование',
 'структура',
 'данные',
 'объектный',
 'ориентировать',
 'программирование',
 'событийный',
 'ориентировать',
 'программировань',
 'специалист',
 'должный',
 'уметь',
 'пользоваться',
 'пользоваться',
 'реализовать',
 'алгоритм',
 'язык',
 'программирование',
 'основной',
 'принцип',
 'понятие',
 'разработка',
 'игра',
 'технология',
 'работа',
 'объект',
 'сцена',
 'компонент',
 'объект',
 'настройка',
 'игровой'

In [202]:
c = dict(Counter(S))
c

{'принцип': 567,
 'навык': 127,
 'обеспечивать': 73,
 'продуктивный': 67,
 'работа': 936,
 'преимущество': 22,
 'использование': 526,
 'современный': 100,
 'средство': 225,
 'разработка': 163,
 'специалист': 410,
 'должный': 443,
 'уметь': 367,
 'использовать': 532,
 'предоставить': 23,
 'ресурс': 46,
 'применять': 230,
 'исследовательский': 22,
 'технология': 290,
 'иметь': 25,
 'представление': 42,
 'самый': 10,
 'последний': 23,
 'отраслевой': 51,
 'рекомендация': 43,
 'анализировать': 46,
 'результат': 132,
 'собственный': 48,
 'деятельность': 186,
 'сравнение': 12,
 'ожидание': 38,
 'потребность': 73,
 'клиент': 236,
 'организация': 183,
 'базовый': 39,
 'алгоритм': 28,
 'программирование': 57,
 'структура': 81,
 'данные': 107,
 'объектный': 12,
 'ориентировать': 18,
 'событийный': 5,
 'программировань': 1,
 'пользоваться': 30,
 'реализовать': 9,
 'язык': 48,
 'основной': 175,
 'понятие': 16,
 'игра': 8,
 'объект': 111,
 'сцена': 7,
 'компонент': 142,
 'настройка': 58,
 'игровой':

In [203]:
d2 = dict((k, v) for k, v in c.items() if v <= 100)
d2

{'обеспечивать': 73,
 'продуктивный': 67,
 'преимущество': 22,
 'современный': 100,
 'предоставить': 23,
 'ресурс': 46,
 'исследовательский': 22,
 'иметь': 25,
 'представление': 42,
 'самый': 10,
 'последний': 23,
 'отраслевой': 51,
 'рекомендация': 43,
 'анализировать': 46,
 'собственный': 48,
 'сравнение': 12,
 'ожидание': 38,
 'потребность': 73,
 'базовый': 39,
 'алгоритм': 28,
 'программирование': 57,
 'структура': 81,
 'объектный': 12,
 'ориентировать': 18,
 'событийный': 5,
 'программировань': 1,
 'пользоваться': 30,
 'реализовать': 9,
 'язык': 48,
 'понятие': 16,
 'игра': 8,
 'сцена': 7,
 'настройка': 58,
 'игровой': 31,
 'работать': 90,
 'верстать': 2,
 'оптимизация': 23,
 'компьютерный': 42,
 'приложение': 100,
 'пк': 9,
 'мобильный': 28,
 'архитектура': 27,
 'контекст': 15,
 'оптимизировать': 28,
 'текстура': 36,
 'целевой': 55,
 'платформа': 29,
 'физика': 13,
 'умение': 43,
 'встроить': 22,
 'движок': 26,
 'внутренний': 29,
 'внешний': 65,
 'профайлер': 6,
 'принятие': 28,


In [213]:
dictonary = list(d2.keys())

### Создание нового атибута на основе словаря

In [248]:
data['Словарь'] = None

In [249]:
for i, sent in enumerate(data["Умения"]):
    sent = sent.replace('•','')
    sent = sent.replace('\\','')
    words = lemmatize(sent)
    test_data.loc[i, 'Словарь'] = str(list(set(dictonary) & set(words)))     

In [250]:
data 

,Компетенция,Показатель,Процент,Умения,Словарь
0,Разработка компьютерных игр и мультимедийных п...,Организация и управление работой,13,"• Принципы и навыки, обеспечивающие продуктив...","['ожидание', 'рекомендация', 'последний', 'сов..."
1,Разработка компьютерных игр и мультимедийных п...,Программирование на языке С#,27,• Базовые алгоритмы программирования • Структ...,"['объектный', 'язык', 'структура', 'событийный..."
2,Разработка компьютерных игр и мультимедийных п...,Unity 3D,30,• Основные принципы и понятия разработки игр ...,"['игровой', 'сцена', 'настройка', 'игра', 'пон..."
3,Разработка компьютерных игр и мультимедийных п...,Оптимизация компьютерных игр и приложений,10,• Особенности оптимизации компьютерных игр и ...,"['движок', 'приложение', 'платформа', 'архитек..."
4,Разработка компьютерных игр и мультимедийных п...,Анализ и проектирование,15,• Важность принятия во внимание всех возможны...,"['суждение', 'системный', 'приложение', 'приня..."
...,...,...,...,...,...
931,Холодильная техника и системы кондиционировани...,Коммуникация,5,• личности и рабочие функции членов строитель...,"['работник', 'компетенция', 'понятие', 'печатн..."
932,Холодильная техника и системы кондиционировани...,Проектирование систем ХС и КВ,15,"• стандартные единицы измерений, используемые...","['свойство', 'тепло', 'хс', 'жидкость', 'измер..."
933,Холодильная техника и системы кондиционировани...,Монтаж и техобслуживание систем ХС и КВ,30,"• нормы техники безопасности и охраны труда, ...","['ремонт', 'тестирование', 'техобслуживание', ..."
934,Холодильная техника и системы кондиционировани...,Ввод в эксплуатацию системы ХС и КВ,20,• знание основ и последовательности пусконала...,"['температура', 'знание', 'электропитание', 'н..."
